In [ ]:
from Bio import SeqIO
import re
import csv
import duckdb

# internal representation
from protein_db import ProteinDB

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)

res = con.execute("DROP TABLE PFAM_TOKEN")

con.close()

In [ ]:

'''
con = duckdb.connect(database=ProteinDB.db_string)

con.execute("\
    CREATE TABLE PFAM_TOKEN(\
        UNIPROT_ID VARCHAR,\
        TOKEN VARCHAR,\
        START_POS USMALLINT,\
        END_POS USMALLINT\
    )")

con.close()
'''

In [ ]:
# this is instantaneous but not sure where it goes or how to query
file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/pfam_entries_full.dat"
duckdb.read_csv(file)

In [ ]:
# THIS WORKS 0 CREATES A TABLE WITH COLUMNS : column0, column1 etc
# 
# this took 2mins 17s to load 300M entries - 298,766,058
#
#
con = duckdb.connect(database=ProteinDB.db_string)           

con.execute("CREATE TABLE PFAM_TOKEN AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/pfam_entries_full.dat')")

con.close()

In [ ]:
# describe the table
con = duckdb.connect(database=ProteinDB.db_string)           

res = con.execute("DESCRIBE PFAM_TOKEN").fetchall()

print(res)

con.close()

In [ ]:
# run a query
con = duckdb.connect(database=ProteinDB.db_string)           
count = con.execute("SELECT COUNT(*) FROM PFAM_TOKEN").fetchall()
print(count)
con.close()

In [ ]:
# create index
con = duckdb.connect(database=ProteinDB.db_string)      
ind = con.execute("create index pfam_idx on PFAM_TOKEN (column0)").fetchall()
print(ind)
con.close()

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)      
res1 = con.execute("select column0, column1, column2, column3 from PFAM_TOKEN where column0='A0A000'").fetchall()
res2 = con.execute("select column0, column1, column2, column3 from PFAM_TOKEN where column0='Z9JZ56'").fetchall()
print(res1)
print(res2)
con.close()

# OLD

In [ ]:
#
# Try a dictionary - also seemed slow
#
MAX_COUNT   = -1
OUTPUT_LIMIT = 100000

input = "/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/pfam_entries_full.dat"

def load_pfam_db():
    count  = 0
    con = duckdb.connect(database=ProteinDB.db_string) 
    
    with open(input) as csvfile:
        iprreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in iprreader:
            
            uniprot_id  = row[0]
            pfam_code   = row[1]
            start       = row[2]
            end         = row[3]
            
            print(uniprot_id, pfam_code, start, end)
            
            #con.execute("INSERT INTO PFAM_TOKEN (UNIPROT_ID, TOKEN, START_POS, END_POS) VALUES(?,?,?,?)", (uniprot_id, pfam_code, start, end))
            count += 1
            
            if (count % OUTPUT_LIMIT == 0):
                print(count, 'lines processed.....')
                
             # -------- check for termination ------------
            if(MAX_COUNT != -1):
                if count >= MAX_COUNT:
                    print('Processing limit reached %s stopping.' % (MAX_COUNT))
                    return
            # --------------------------------------------
    con.close()
    
load_pfam_db()

In [ ]:
MAX_COUNT   = 1000000
OUTPUT_LIMIT = 1000

input = "/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/pfam_entries_full.dat"

def load_pfam_dict():
    count  = 0
    proteins={}
    
    with open(input) as csvfile:
        iprreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in iprreader:
            uniprot_id  = row[0]
            pfam_code   = row[1]
            start       = row[2]
            end         = row[3]
            
            print('processing', uniprot_id, pfam_code, start, end)
            if(uniprot_id in proteins.keys()):
                #print(uniprot_id, 'already found')
                #print(proteins[uniprot_id])
                proteins[uniprot_id].append({pfam_code, start, end})
            else:
                #print('New entry:', uniprot_id)
                proteins[uniprot_id] = [{pfam_code, start, end}]
            #print(uniprot_id, pfam_code, start, end)
            count +=1
            
            if (count % OUTPUT_LIMIT == 0):
                print(count, 'lines processed.....')
                
             # -------- check for termination ------------
            if(MAX_COUNT != -1):
                if count >= MAX_COUNT:
                    print('Processing limit reached %s stopping.' % (MAX_COUNT))
                    #print(proteins) 
                    return
            # --------------------------------------------
    print('keys:', proteins)    
load_pfam_dict()